In [3]:
import pandas as pd
import os
import csv

In [43]:
def NormalizeStat(stat, min_, max_):
    return (stat - min_) / (max_ - min_)

In [36]:
df_game_log = pd.read_csv(os.path.join("Resources", "game_logs.csv"))
df_players = pd.read_csv(os.path.join("Resources", "players.csv"))

df_players = df_players.rename(columns={'id':'player_id'})

df_player_game_logs = df_game_log.merge(df_players, how='inner', on='player_id')

df_player_game_logs = df_player_game_logs.set_index('year')

df_player_game_logs.head()

,player_id,date,week,team_x,location,opponent,result,started,rushing_attempts,rushing_yards,...,receptions,receiving_yards,receiving_touchdowns,conversions,fumbles_lost,fumble_touchdowns,name,team_y,team_abbr,position
year,,,,,,,,,,,,,,,,,,,,,
2017,KamaAl00,2017-09-11,1,NOR,Away,MIN,L,False,7,18,...,4,20,0,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB
2017,KamaAl00,2017-09-17,2,NOR,Home,NWE,L,False,1,3,...,3,51,0,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB
2017,KamaAl00,2017-09-24,3,NOR,Away,CAR,W,False,2,37,...,3,5,0,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB
2017,KamaAl00,2017-10-01,4,NOR,Away,MIA,W,False,5,25,...,10,71,1,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB
2017,KamaAl00,2017-10-15,6,NOR,Home,DET,W,True,10,75,...,3,12,0,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB


In [46]:
df_2020_running_backs = df_player_game_logs.loc[df_player_game_logs.index.isin([2020]) & (df_player_game_logs.position == 'RB')]

df_2020_running_backs['rushing_yard_rank'] = [NormalizeStat(r, min(df_2020_running_backs['rushing_yards']), max(df_2020_running_backs['rushing_yards'])) for r in df_2020_running_backs['rushing_yards']]
df_2020_running_backs['rushing_td_rank'] = [NormalizeStat(r, min(df_2020_running_backs['rushing_touchdowns']), max(df_2020_running_backs['rushing_touchdowns'])) for r in df_2020_running_backs['rushing_touchdowns']]

df_2020_running_backs.eval("overall_ranking = rushing_yard_rank + rushing_td_rank", inplace=True)

df_grouped_game_log = df_2020_running_backs.groupby(['name', 'year'])

print(df_grouped_game_log['rushing_yards'].sum().nlargest(5),
df_grouped_game_log['overall_ranking'].sum().nlargest(10))

name           year
Dalvin Cook    2020    489
Josh Jacobs    2020    377
Derrick Henry  2020    376
Todd Gurley    2020    375
Aaron Jones    2020    374
Name: rushing_yards, dtype: int64 name             year
Dalvin Cook      2020    5.177893
Josh Jacobs      2020    3.930915
Todd Gurley      2020    3.920553
Ezekiel Elliott  2020    3.863558
Derrick Henry    2020    3.530225
Aaron Jones      2020    3.519862
Nick Chubb       2020    3.317789
Kareem Hunt      2020    3.108808
Alvin Kamara     2020    3.100173
James Robinson   2020    3.036269
Name: overall_ranking, dtype: float64


In [68]:
df_2020_pit_wide_receivers = df_player_game_logs.loc[df_player_game_logs.index.isin([2020]) & (df_player_game_logs.position == 'WR') & (df_player_game_logs.team_abbr == 'PIT')]

df_2020_pit_wide_receivers['receiving_touchdown_rank'] = [NormalizeStat(r, min(df_2020_pit_wide_receivers['receiving_touchdowns']), max(df_2020_pit_wide_receivers['receiving_touchdowns'])) for r in df_2020_pit_wide_receivers['receiving_touchdowns']]

df_2020_pit_wide_receivers_grouped = df_2020_pit_wide_receivers.groupby(['name'])

print(df_2020_pit_wide_receivers_grouped['receiving_touchdown_rank'].sum().nlargest(10))

name
Chase Claypool         1.333333
JuJu Smith-Schuster    1.000000
Diontae Johnson        0.333333
James Washington       0.333333
Ray-Ray McCloud        0.000000
Name: receiving_touchdown_rank, dtype: float64


In [69]:
df_2020_game_logs = df_player_game_logs.loc[df_player_game_logs.index.isin([2020])]
print(df_2020_game_logs.columns)

df_2020_groupby_opponent = df_2020_game_logs.groupby('opponent')

df_2020_groupby_opponent[['rushing_attempts', 'rushing_yards', 'rushing_touchdowns']].sum().sort_values(by='rushing_touchdowns', ascending=False)

Index(['player_id', 'date', 'week', 'team_x', 'location', 'opponent', 'result',
       'started', 'rushing_attempts', 'rushing_yards', 'rushing_touchdowns',
       'pass_targets', 'receptions', 'receiving_yards', 'receiving_touchdowns',
       'conversions', 'fumbles_lost', 'fumble_touchdowns', 'name', 'team_y',
       'team_abbr', 'position'],
      dtype='object')


,rushing_attempts,rushing_yards,rushing_touchdowns
opponent,,,
LVR,131,634,10
NYJ,154,643,9
WAS,153,648,8
MIA,127,630,8
PHI,145,571,8
CAR,123,667,8
DAL,167,779,7
BUF,126,543,7
NYG,149,553,6
